In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [3]:
#import data set
df=pd.read_csv('IPL_ball_by_ball_updated.csv')

In [4]:
df['total_runs']=df['runs_off_bat']+df['extras']
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,total_runs
0,335982,2008,18-04-2008,M Chinnaswamy Stadium,2,6.8,Royal Challengers Bangalore,Kolkata Knight Riders,MV Boucher,CL White,AB Agarkar,4,0,0,0,0,0,0,NaN,NaN,NaN,NaN,4
1,335982,2008,18-04-2008,M Chinnaswamy Stadium,2,2.7,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,JH Kallis,AB Dinda,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1
2,335982,2008,18-04-2008,M Chinnaswamy Stadium,2,3.1,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,JH Kallis,I Sharma,2,0,0,0,0,0,0,NaN,NaN,NaN,NaN,2
3,335982,2008,18-04-2008,M Chinnaswamy Stadium,2,3.2,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,JH Kallis,I Sharma,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1
4,335982,2008,18-04-2008,M Chinnaswamy Stadium,2,3.3,Royal Challengers Bangalore,Kolkata Knight Riders,JH Kallis,W Jaffer,I Sharma,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0


In [5]:
matches =list(set(df.match_id.tolist()))
len(matches)  ## total matches 876.

876

In [6]:
first_inn_df = df[df.innings==1]
m1=list(set(first_inn_df.match_id.tolist()))
len(m1)  ## 876 matches in 1st innings.

876

In [7]:
second_inn_df = df[df.innings==2]
m2=list(set(second_inn_df.match_id.tolist()))
len(m2)  ## 874 matches in 2nd innings. 2 matches data missing 

874

In [8]:
innings_2_missing = [i for i in matches if i not in m2] #matches where 2nd innings data is not ther because of No result
innings_2_missing

[829763, 501265]

In [9]:
#function to get ScoreCard for one match

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

def ipl(matchid, inning):
    #df=pd.read_csv('ipl_ball_by_ball_data.csv')
    
    req_df=df[(df.match_id==matchid)&(df.innings== inning)]     
    
    ndf=req_df.sort_values('ball')   #sort the innings data by balls        
    a=ndf.striker.unique()           #Unique names of striker in correct order
    b=ndf.non_striker.unique()       #Unique names of non-striker in correctorder
       
    combine_list=[]                 #empty list, later used to combine striker and non striker.
    for aval, bval in zip(a, b):    #zip a and b together and append in order
        combine_list.append(aval)
        combine_list.append(bval) 
        
    def bat_order(list1):         #function to return correct batting order based on batting position
        proper_list = []   
        for x in combine_list:

            if x not in proper_list:
                proper_list.append(x)

        return (proper_list)      

    fdf=bat_order(combine_list)  #function call
    

    bat_order_df=pd.DataFrame(fdf).rename(columns={0:'Batsman'})  #convert the result into data frame
    
    bat_order_df['Bat_order'] = range(1, len(bat_order_df)+1)  # add batting position
    
    
    ndf['IsOne']=ndf['runs_off_bat'].apply(lambda x: 1 if x==1 else 0)
    ndf['IsTwo']=ndf['runs_off_bat'].apply(lambda x: 1 if x==2 else 0)
    ndf['IsThree']=ndf['runs_off_bat'].apply(lambda x: 1 if x==3 else 0)
    ndf['IsFour']=ndf['runs_off_bat'].apply(lambda x: 1 if x==4 else 0)
    ndf['IsSix']=ndf['runs_off_bat'].apply(lambda x: 1 if x==6 else 0)
    runs_df = pd.DataFrame(ndf.groupby(['season','match_id','innings','striker'])['runs_off_bat','IsFour','IsSix'].sum()).reset_index().rename(columns={'striker':'Batsman','runs_off_bat':'Runs','IsFour':'Fours','IsSix':'Sixes'})
    
    #create a data frame for runs
    
    req_df['balls_faced']=req_df['wides'].apply(lambda x: 1 if x==0 else 0)
    balls_df = pd.DataFrame(req_df.groupby(['striker'])['balls_faced'].sum()).reset_index().rename(columns={'striker':'Batsman','runs_off_bat':'Runs'})

    
    bat_runs_df = bat_order_df.merge(runs_df, on = 'Batsman', how = 'left')  #merge bat order and runs data frame
    
    final_df = bat_runs_df.merge(balls_df, on = 'Batsman', how = 'left')
    
    order_df=final_df.loc[:, ["season","match_id","innings","Bat_order","Batsman","Runs","balls_faced","Fours","Sixes"]]  #arrange the columns
    
    order_df['SR']=round((order_df['Runs']/order_df['balls_faced'])*100,2)
    
    return order_df  #return the result

ipl(matchid = 1082625 , inning = 2)  #main function call

,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
0,2017,1082625,2,1,PA Patel,70,44,9,1,159.09
1,2017,1082625,2,2,JC Buttler,9,7,2,0,128.57
2,2017,1082625,2,3,N Rana,19,16,1,1,118.75
3,2017,1082625,2,4,RG Sharma,5,13,0,0,38.46
4,2017,1082625,2,5,KA Pollard,15,11,2,0,136.36
5,2017,1082625,2,6,KH Pandya,29,20,2,1,145.00
6,2017,1082625,2,7,HH Pandya,4,5,0,0,80.00
7,2017,1082625,2,8,Harbhajan Singh,0,1,0,0,0.00
8,2017,1082625,2,9,MJ McClenaghan,1,1,0,0,100.00
9,2017,1082625,2,10,JJ Bumrah,0,1,0,0,0.00


In [10]:
first_match = 335982  #to initialise and set up df to append.
inn=[1,2] #innings 1,2

sc_df = ipl(matchid = first_match , inning = 1)
sc_in2=ipl(matchid = first_match , inning = 2)

print(sc_df)
print(sc_in2)

sc_df=sc_df.append(sc_in2).reset_index(drop=True)
print(sc_df)

m_ids = []

for i in m2:
    if i != first_match:
        m_ids.append(i)
        
print(len(matches))        
print(len(m_ids))
print(inn)

   season  match_id  innings  Bat_order          Batsman  Runs  balls_faced  Fours  Sixes      SR
0  2008    335982    1        1          SC Ganguly       10    12           2      0      83.33 
1  2008    335982    1        2          BB McCullum      158   73           10     13     216.44
2  2008    335982    1        3          RT Ponting       20    20           1      1      100.00
3  2008    335982    1        4          DJ Hussey        12    12           1      0      100.00
4  2008    335982    1        5          Mohammad Hafeez  5     3            1      0      166.67
    season  match_id  innings  Bat_order     Batsman  Runs  balls_faced  Fours  Sixes      SR
0   2008    335982    2        1          R Dravid    2     3            0      0      66.67 
1   2008    335982    2        2          W Jaffer    6     16           0      0      37.50 
2   2008    335982    2        3          V Kohli     1     5            0      0      20.00 
3   2008    335982    2        4    

In [11]:
#this is it!!
#for all the matches that has data for innings 1 and 2

for i in m_ids:
    #for count,ele in enumerate(m_ids):
#     if count>100:
#         print (count,ele)
    #print(i)
    for j in inn:
        #print(j)
        dfm=ipl(matchid = i , inning = j)
        #print(dfm)
            #print(dfm)
        sc_df= sc_df.append(dfm).reset_index(drop=True)
            #print(sc)

sc_df.tail(20)

,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
12976,2013.0,598014.0,2.0,2,AM Rahane,34.0,42.0,3.0,0.0,80.95
12977,2013.0,598014.0,2.0,3,R Dravid,9.0,12.0,2.0,0.0,75.00
12978,2013.0,598014.0,2.0,4,STR Binny,0.0,4.0,0.0,0.0,0.00
12979,2013.0,598014.0,2.0,5,BJ Hodge,15.0,16.0,2.0,0.0,93.75
12980,2013.0,598014.0,2.0,6,SV Samson,27.0,23.0,3.0,0.0,117.39
12981,2013.0,598015.0,1.0,1,RV Uthappa,26.0,33.0,2.0,0.0,78.79
12982,2013.0,598015.0,1.0,2,AJ Finch,67.0,45.0,10.0,2.0,148.89
12983,2013.0,598015.0,1.0,3,LRPL Taylor,8.0,13.0,0.0,0.0,61.54
12984,2013.0,598015.0,1.0,4,SPD Smith,39.0,16.0,3.0,3.0,243.75
12985,2013.0,598015.0,1.0,5,MR Marsh,2.0,4.0,0.0,0.0,50.00


In [12]:
len(sc_df)

12996

In [13]:
#this is it!!
#matches with no second innings data
for i in innings_2_missing:        
    dfm=ipl(matchid = i , inning = 1)

    sc_df= sc_df.append(dfm).reset_index(drop=True)


sc_df.tail(20)

,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
12989,2013.0,598015.0,2.0,3,SK Raina,8.0,10.0,1.0,0.0,80.00
12990,2013.0,598015.0,2.0,4,S Badrinath,34.0,26.0,4.0,0.0,130.77
12991,2013.0,598015.0,2.0,5,RA Jadeja,27.0,22.0,1.0,1.0,122.73
12992,2013.0,598015.0,2.0,6,MS Dhoni,10.0,13.0,0.0,0.0,76.92
12993,2013.0,598015.0,2.0,7,DJ Bravo,1.0,4.0,0.0,0.0,25.00
12994,2013.0,598015.0,2.0,8,JA Morkel,13.0,11.0,0.0,1.0,118.18
12995,2013.0,598015.0,2.0,9,CH Morris,5.0,5.0,0.0,0.0,100.00
12996,2015.0,829763.0,1.0,1,CH Gayle,10.0,4.0,1.0,1.0,250.00
12997,2015.0,829763.0,1.0,2,V Kohli,1.0,4.0,0.0,0.0,25.00
12998,2015.0,829763.0,1.0,3,AB de Villiers,57.0,45.0,9.0,1.0,126.67


In [13]:
# def scorecard(match_id):
#     return sc_df[sc_df.match_id==match_id]

# scorecard(548362)

,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
2453,2012.0,548362.0,1.0,1,CH Gayle,57.0,31.0,3.0,6.0,183.87
2454,2012.0,548362.0,1.0,2,TM Dilshan,53.0,44.0,6.0,1.0,120.45
2455,2012.0,548362.0,1.0,3,V Kohli,9.0,12.0,1.0,0.0,75.00
2456,2012.0,548362.0,1.0,4,SS Tiwary,36.0,30.0,3.0,1.0,120.00
2457,2012.0,548362.0,1.0,5,AB de Villiers,9.0,4.0,0.0,1.0,225.00
2458,2012.0,548362.0,2.0,1,MD Mishra,4.0,4.0,1.0,0.0,100.00
2459,2012.0,548362.0,2.0,2,MJ Clarke,13.0,10.0,2.0,0.0,130.00
2460,2012.0,548362.0,2.0,3,MK Pandey,0.0,2.0,0.0,0.0,0.00
2461,2012.0,548362.0,2.0,4,RV Uthappa,38.0,23.0,5.0,1.0,165.22
2462,2012.0,548362.0,2.0,5,AP Majumdar,31.0,26.0,4.0,0.0,119.23


In [14]:
sc_df[sc_df.Batsman=='MS Dhoni'].sort_values('season') 

,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
1362,2008.0,336005.0,1.0,6,MS Dhoni,1.0,7.0,0.0,0.0,14.29
1863,2008.0,336040.0,1.0,5,MS Dhoni,29.0,17.0,1.0,2.0,170.59
1822,2008.0,336037.0,2.0,4,MS Dhoni,37.0,25.0,4.0,1.0,148.00
1760,2008.0,336033.0,2.0,5,MS Dhoni,12.0,8.0,2.0,0.0,150.00
1716,2008.0,336029.0,2.0,3,MS Dhoni,4.0,8.0,0.0,0.0,50.00
1557,2008.0,336018.0,1.0,6,MS Dhoni,43.0,35.0,6.0,1.0,122.86
1477,2008.0,336013.0,1.0,5,MS Dhoni,60.0,43.0,2.0,4.0,139.53
1427,2008.0,336009.0,2.0,3,MS Dhoni,33.0,33.0,3.0,0.0,100.00
1393,2008.0,336007.0,1.0,6,MS Dhoni,23.0,31.0,2.0,0.0,74.19
1296,2008.0,336001.0,1.0,4,MS Dhoni,32.0,27.0,2.0,1.0,118.52


In [15]:
ms_df_bo=pd.DataFrame(sc_df[sc_df.Batsman=='MS Dhoni'].sort_values('season')).reset_index()

In [16]:
ms_df_bo

,index,season,match_id,innings,Bat_order,Batsman,Runs,balls_faced,Fours,Sixes,SR
0,1362,2008.0,336005.0,1.0,6,MS Dhoni,1.0,7.0,0.0,0.0,14.29
1,1863,2008.0,336040.0,1.0,5,MS Dhoni,29.0,17.0,1.0,2.0,170.59
2,1822,2008.0,336037.0,2.0,4,MS Dhoni,37.0,25.0,4.0,1.0,148.00
3,1760,2008.0,336033.0,2.0,5,MS Dhoni,12.0,8.0,2.0,0.0,150.00
4,1716,2008.0,336029.0,2.0,3,MS Dhoni,4.0,8.0,0.0,0.0,50.00
5,1557,2008.0,336018.0,1.0,6,MS Dhoni,43.0,35.0,6.0,1.0,122.86
6,1477,2008.0,336013.0,1.0,5,MS Dhoni,60.0,43.0,2.0,4.0,139.53
7,1427,2008.0,336009.0,2.0,3,MS Dhoni,33.0,33.0,3.0,0.0,100.00
8,1393,2008.0,336007.0,1.0,6,MS Dhoni,23.0,31.0,2.0,0.0,74.19
9,1296,2008.0,336001.0,1.0,4,MS Dhoni,32.0,27.0,2.0,1.0,118.52


In [17]:
ms_df_bo.to_csv('MSD_data_batting order.csv')
sc_df.to_csv('IPL_mbm_sc')